# News Clustering using KMeans Algorithm

In [1]:
import numpy as np
import pandas as pd
from time import time
import articles_data_py3 as articles_data
from pymongo import MongoClient
import datetime

## Load data from MongoDB 

In [2]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('articles')

## Select Categories
- 포함 : 경제, 문화, 건강, 과학, 사회, 정치, 스포츠
- 제외 : 종합, 정보없음, 인물, 사설

In [3]:
catelist_path = '../datastore/category.p'
headline_path = '../datastore/headline.p'
target_time = datetime.datetime.now()

In [4]:
train = articles_data.find_recent_articles(crawled_collection, catelist_path, target_time)

NameError: name 'is_dirty_article' is not defined

## Preprocessing
1. Datetime (16-09-11 ~ 16-09-17)
2. Remove stopwords (regex, hanja)
3. POS Tagging with KoNLPy, Mecab
4. Using bigram

In [ ]:
from konlpy.tag import Mecab
import cnouns
import hanja
import re

In [ ]:
mecab = Mecab()

In [ ]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]

In [ ]:
train['title_flat'] = train['title'].apply(lambda text: articles_data.remove_headlines(text, headline_path))
train['title_flat'] = train['title_flat'].apply(lambda text: cnouns.text_cleaning_without_special_ch(text))
title = [tokenize(each[1]['title_flat']) for each in train.iterrows()]

## Training
1. Feature extraction - TfidVectorizer
2. Decomposition - PCA
3. Cluster - KMeans

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)

In [ ]:
x_list_100d = PCA(n_components=100).fit_transform(x_list.toarray())
x_list_100d.shape

### Scoring

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Best Silhoutte Score

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
best_score = 0.0
best_k = 0

In [ ]:
k_list = []
score_list = []
for k in range(20, 35):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list_100d)
    score = silhouette_score(x_list_100d, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    k_list.append(k)
    score_list.append(score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In [ ]:
x = k_list
y = score_list
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette score')
plt.title('Finding best k')
plt.plot(x, y)
plt.show()

### K-Means Algorithm

In [ ]:
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list_100d)
labels = km.labels_
centroids = km.cluster_centers_
print(km.inertia_)

In [ ]:
x_list_vector = x_list_100d.tolist()
train = train.drop(['title_flat'], axis=1)
train['vector'] = x_list_vector
train['cluster'] = labels

## Choose Best Cluster
- Minimum inertia

In [ ]:
sample_silhouette_values = silhouette_samples(x_list_100d, labels)
sample_silhouette_score = []
list_silhouette_score = []
best_cluster = []

for i in range(best_k):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]    
    list_silhouette_score.append(abs(ith_cluster_silhouette_values.mean()))

list_silhouette_score.sort(reverse=True)
sample_silhouette_score = list_silhouette_score[:12]

In [ ]:
x = [x for x in range(best_k)]
y = list_silhouette_score
width = 1/1.5
plt.xlabel('Each of Cluster')
plt.ylabel('Silhouette score')
plt.title('Finding best cluster')
plt.bar(x, y, width)
plt.show()

In [ ]:
sample_silhouette_score

In [ ]:
cohesions = []

In [ ]:
for i in range(best_k):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    if abs(ith_cluster_silhouette_values.mean()) in sample_silhouette_score:
        best_cluster.append(i)
        cohesions.append(abs(ith_cluster_silhouette_values.mean()))

In [ ]:
train = train[train['cluster'].isin(best_cluster)]
cohesions

## Save to MongoDB

In [ ]:
articles_data.save_to_db(train, 1108, clusters_collection, cohesions)
client.close()

## Result

In [ ]:
cluster_data = []

for cluster_index in range(best_k):
    if cluster_index in best_cluster:
        cluster_data.append(train[train['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(len(train.cluster.unique()), len(d))))
    print('\n\n')